# Generación de archivos parquet SEGCOVID

## Carga de los datos

Este jupyter notebook esta generando con todos los datos de SEGCOVID (2020 - 2023)

In [1]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("GeneraciónTablasSEGCOVID_Todos").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/23 10:03:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#spark.stop() ruta = "/rawdata/rips/rips_parquet_*/"

In [3]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/segcovid/segcovid_parquet_*/")

df=df1_parquet

In [4]:
# Contar el número de registros en el DataFrame unido
total_registros = df.count()

# Listar las columnas del DataFrame unido
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 18431733
Columnas del DataFrame unido: ['PersonaBasicaID', 'AntecedenteVIH', 'AntecedenteHipertension', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana', 'AntecedenteCancer', 'AntecedenteArtritis', 'AntecedenteERC', 'AntecedenteEPOC', 'AntecedenteAsma', 'AntecedenteObesidad', 'FactorRiesgoMayor59', 'FactorRiesgoDiabetes', 'FactorRiesgoEnfermedadCardiovascular', 'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoCancer', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoCondicionImpidaAislamiento', 'FechaIngresoAtencion', 'FechaEgresoAtencion', 'FechaRegistro', 'FechaSeguimiento', 'DepartamentoAtencion', 'MunicipioAtencion', 'AmbitoAtencion', 'SignosAlarmaRespiracionRapida', 'SignosAlarmaFiebre', 'SignosAlarmaPechoSuena', 'SignosAlarmaSomnolencia', 'SignosAlarmaConvulsiones', 'SignosAlarmaDecaimiento', 'SignosAlarmaDeterioro', 'SintomasFiebre', 'SintomasTos', 'SintomasFatiga', 'SintomasDificultadRespirar', 'SintomasDolorGarganta', 'Sint

In [5]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regIstros del año 2022 (128.343)
record_count = df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 18431733


In [6]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- PersonaBasicaID: string (nullable = true)
 |-- AntecedenteVIH: string (nullable = true)
 |-- AntecedenteHipertension: string (nullable = true)
 |-- AntecedenteDiabetes: string (nullable = true)
 |-- AntecedenteEnfermedadHuerfana: string (nullable = true)
 |-- AntecedenteCancer: string (nullable = true)
 |-- AntecedenteArtritis: string (nullable = true)
 |-- AntecedenteERC: string (nullable = true)
 |-- AntecedenteEPOC: string (nullable = true)
 |-- AntecedenteAsma: string (nullable = true)
 |-- AntecedenteObesidad: string (nullable = true)
 |-- FactorRiesgoMayor59: string (nullable = true)
 |-- FactorRiesgoDiabetes: string (nullable = true)
 |-- FactorRiesgoEnfermedadCardiovascular: string (nullable = true)
 |-- FactorRiesgoEnfermedadRespiratoriaCronica: string (nullable = true)
 |-- FactorRiesgoCancer: string (nullable = true)
 |-- FactorRiesgoInmunodeficiencia: string (nullable = true)
 |-- FactorRiesgoCondicionImpidaAislamiento: string (nullable = true)
 |-- FechaIngresoAt

# Empieza el proceso de generación de los datos agrupados por atenciones

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
#spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH',
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' 
   
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('PersonaBasicaID')
df_filtered = df_filtered.withColumn('personaid_count', F.count('PersonaBasicaID').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
print("Nombres de las columnas filtradas:")
print(df_filtered.columns)

# Agrupar por la columna 'PersonaBasicaID'
grouped_df = df_filtered.groupBy('PersonaBasicaID').agg(
    F.first('FechaRegistro').alias('FechaRegistro'),
    F.first('MunicipioAtencion').alias('MunicipioAtencion'),
    F.first('AntecedenteHipertension').alias('AntecedenteHipertension'),
    F.first('AntecedenteArtritis').alias('AntecedenteArtritis'),
    F.first('AntecedenteAsma').alias('AntecedenteAsma'),
    F.first('AntecedenteCancer').alias('AntecedenteCancer'),
    F.first('AntecedenteDiabetes').alias('AntecedenteDiabetes'),
    F.first('AntecedenteEnfermedadHuerfana').alias('AntecedenteEnfermedadHuerfana'),
    F.first('AntecedenteEPOC').alias('AntecedenteEPOC'),
    F.first('AntecedenteERC').alias('AntecedenteERC'),
    F.first('AntecedenteObesidad').alias('AntecedenteObesidad'),
    F.first('AntecedenteVIH').alias('AntecedenteVIH'),
    
    F.first('FactorRiesgoCancer').alias('FactorRiesgoCancer'),
    F.first('FactorRiesgoCondicionImpidaAislamiento').alias('FactorRiesgoCondicionImpidaAislamiento'),
    F.first('FactorRiesgoDiabetes').alias('FactorRiesgoDiabetes'),
    F.first('FactorRiesgoEnfermedadCardiovascular').alias('FactorRiesgoEnfermedadCardiovascular'),
    F.first('FactorRiesgoEnfermedadRespiratoriaCronica').alias('FactorRiesgoEnfermedadRespiratoriaCronica'),
    F.first('FactorRiesgoInmunodeficiencia').alias('FactorRiesgoInmunodeficiencia'),
    F.first('FactorRiesgoMayor59').alias('FactorRiesgoMayor59'),
    
    
    F.first('personaid_count').alias('personaid_count')
)
# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


+---------------+-------------+-------------------+-----------------------+-------------------+---------------+-----------------+-------------------+-----------------------------+---------------+--------------+-------------------+--------------+------------------+--------------------------------------+--------------------+------------------------------------+-----------------------------------------+-----------------------------+-------------------+---------------+
|PersonaBasicaID|FechaRegistro|  MunicipioAtencion|AntecedenteHipertension|AntecedenteArtritis|AntecedenteAsma|AntecedenteCancer|AntecedenteDiabetes|AntecedenteEnfermedadHuerfana|AntecedenteEPOC|AntecedenteERC|AntecedenteObesidad|AntecedenteVIH|FactorRiesgoCancer|FactorRiesgoCondicionImpidaAislamiento|FactorRiesgoDiabetes|FactorRiesgoEnfermedadCardiovascular|FactorRiesgoEnfermedadRespiratoriaCronica|FactorRiesgoInmunodeficiencia|FactorRiesgoMayor59|personaid_count|
+---------------+-------------+-------------------+---------

24/08/23 10:03:52 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------+-------------+--------------------+-----------------------+-------------------+---------------+-----------------+-------------------+-----------------------------+---------------+--------------+-------------------+--------------+------------------+--------------------------------------+--------------------+------------------------------------+-----------------------------------------+-----------------------------+-------------------+---------------+
|PersonaBasicaID|FechaRegistro|   MunicipioAtencion|AntecedenteHipertension|AntecedenteArtritis|AntecedenteAsma|AntecedenteCancer|AntecedenteDiabetes|AntecedenteEnfermedadHuerfana|AntecedenteEPOC|AntecedenteERC|AntecedenteObesidad|AntecedenteVIH|FactorRiesgoCancer|FactorRiesgoCondicionImpidaAislamiento|FactorRiesgoDiabetes|FactorRiesgoEnfermedadCardiovascular|FactorRiesgoEnfermedadRespiratoriaCronica|FactorRiesgoInmunodeficiencia|FactorRiesgoMayor59|personaid_count|
+---------------+-------------+--------------------+------

In [8]:
# Contar el número de registros - cantidad de registros de todos los años 5'633.748 por id unico
record_count = grouped_df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 5633748


In [9]:
 df_final =grouped_df


In [10]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 5633748


In [9]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show()

+---------------+-------------+--------------------+-----------------------+-------------------+---------------+-----------------+-------------------+-----------------------------+---------------+--------------+-------------------+--------------+---------------+
|PersonaBasicaID|FechaRegistro|   MunicipioAtencion|AntecedenteHipertension|AntecedenteArtritis|AntecedenteAsma|AntecedenteCancer|AntecedenteDiabetes|AntecedenteEnfermedadHuerfana|AntecedenteEPOC|AntecedenteERC|AntecedenteObesidad|AntecedenteVIH|personaid_count|
+---------------+-------------+--------------------+-----------------------+-------------------+---------------+-----------------+-------------------+-----------------------------+---------------+--------------+-------------------+--------------+---------------+
|      100000046|   2021-06-30|      25430 - Madrid|                     No|                 No|             No|               No|                 No|                           No|             No|            No|

# Empieza el proceso de generación de tablas SEGCOVID

# Tablas Antecedentes 

## 1. Agrupado por año, mes, AntecedenteHipertension y MunicipioAtencion

In [12]:
# Cargar el archivo Parquet en un DataFrame ( uno distinto por si deseo comparar)
df2 = df_final

In [11]:
# Selecciona la columna 'FechaRegistro' y muestra sus datos
#df.select('FechaRegistro').show(100, truncate=False)

In [13]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y MunicipioAtencion
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteHipertension','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteHipertension|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------------------+--------------------+------------------+--------------------+
|2021|  1|                     Sí|       52203 - Colón|                14|                  47|
|2021|  5|                     Sí|       05893 - Yondó|                 2|                   4|
|2021|  1|                     No|     47551 - Pivijay|                62|                 231|
|2021|  7|                     No|    68498 - Ocamonte|                 5|                  20|
|2022|  9|                     No|68081 - Barrancab...|                32|                  71|
|2021| 10|                     No| 68001 - Bucaramanga|               777|                4827|
|2022|  6|                     No| 68001 - Bucaramanga|               669|                1555|
|2021|  6|                     No|     9

In [14]:
#se escribe el parquet tabla 1- Agrupado por año, mes, AntecedenteHipertension y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/1.SEGCOVID_Tabla_anio_mes_AntecedenteHipertension_Todos.parquet") 

# 2. Agrupado por año, mes , AntecedenteArtritis y MunicipioAtencion

In [17]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteArtritis
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteArtritis','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+-------------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteArtritis|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+--------------------+------------------+--------------------+
|2021|  1|                 No| 86885 - Villagarzón|                91|                 263|
|2020|  6|                 No|52835 - San Andre...|                24|                  50|
|2020| 11|                 No|25843 - Villa De ...|                30|                  80|
|2021|  2|                 No|     05376 - La Ceja|               278|                1214|
|2022|  2|                 No|      68770 - Suaita|                14|                  24|
|2021| 10|                 No| 68001 - Bucaramanga|               878|                5344|
|2021|  2|                 No|76109 - Buenaventura|               229|                 603|
|2022|  4|                 No|    -1 - No Definido|              1948|          

In [18]:
#se escribe el parquet tabla 2- Agrupado por año, mes, AntecedenteArtritis y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/2.SEGCOVID_Tabla_anio_mes_AntecedenteArtritis_Todos.parquet") 

# 3. Agrupado por año, mes , AntecedenteAsma y MunicipioAtencion

In [19]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteAsma
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteAsma','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+---------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteAsma|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+---------------+--------------------+------------------+--------------------+
|2021|  5|             No|      41078 - Baraya|                 9|                  29|
|2021|  9|             No|      73770 - Suárez|                 2|                   6|
|2021|  1|             No|     17877 - Viterbo|                85|                 395|
|2021|  2|             No|     05376 - La Ceja|               267|                1162|
|2021|  1|             No| 86885 - Villagarzón|                90|                 265|
|2021|  5|             No|     05197 - Cocorná|                34|                  74|
|2020|  6|             No|52835 - San Andre...|                23|                  48|
|2021|  4|             No|       76845 - Ulloa|                16|                  47|
|2021|  3|             No|      

In [20]:
#se escribe el parquet tabla 3- Agrupado por año, mes, AntecedenteAsma y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/3.SEGCOVID_Tabla_anio_mes_AntecedenteAsma_Todos.parquet") 

# 4. Agrupado por año, mes, AntecedenteCancer y MunicipioAtencion

In [21]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteCancer
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteCancer','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteCancer|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------------+--------------------+------------------+--------------------+
|2021|  1|               No|     47551 - Pivijay|                66|                 244|
|2021|  1|               No| 86885 - Villagarzón|                91|                 267|
|2021|  7|               No|    68498 - Ocamonte|                 5|                  20|
|2022|  9|               No|68081 - Barrancab...|                33|                  73|
|2021| 10|               No| 68001 - Bucaramanga|               870|                5347|
|2022|  6|               No| 68001 - Bucaramanga|               748|                1752|
|2021|  6|               No|     91001 - Leticia|                37|                 118|
|2021|  3|               No|       15001 - Tunja|               474|                2703|
|2022|  4|

In [22]:
#se escribe el parquet tabla 4- Agrupado por año, mes, AntecedenteCancer y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/4.SEGCOVID_Tabla_anio_mes_AntecedenteCancer_Todos.parquet") 

# 5. Agrupado por año, mes, AntecedenteDiabetes y MunicipioAtencion

In [23]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteDiabetes
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteDiabetes','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-------------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteDiabetes|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+--------------------+------------------+--------------------+
|2021|  5|                 No|      41078 - Baraya|                 9|                  29|
|2021| 10|                 Sí|   25377 - La Calera|                 1|                  12|
|2021|  9|                 No|      73770 - Suárez|                 2|                   6|
|2021|  1|                 No|     17877 - Viterbo|                82|                 366|
|2021|  2|                 No|     05376 - La Ceja|               269|                1188|
|2020| 11|                 Sí|        76001 - Cali|                55|                 150|
|2021|  1|                 No| 86885 - Villagarzón|                91|                 263|
|2021|  5|                 No|     05197 - Cocorná|                31|          

In [24]:
#se escribe el parquet tabla 5- Agrupado por año, mes, AntecedenteDiabetes y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/5.SEGCOVID_Tabla_anio_mes_AntecedenteDiabetes_Todos.parquet") 


# 6. Agrupado por año, mes, AntecedenteEnfermedadHuerfana y MunicipioAtencion

In [25]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteEnfermedadHuerfana
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteEnfermedadHuerfana','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------------------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteEnfermedadHuerfana|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------------------------+--------------------+------------------+--------------------+
|2021|  5|                           No|      41078 - Baraya|                 9|                  29|
|2021|  9|                           No|      73770 - Suárez|                 1|                   4|
|2021|  1|                           No|     17877 - Viterbo|                87|                 411|
|2021|  2|                           No|     05376 - La Ceja|               276|                1206|
|2021|  1|                           No| 86885 - Villagarzón|                92|                 268|
|2021|  5|                           No|     05197 - Cocorná|                33|                  67|
|2020|  6|                           No|52835 - San Andre...|                25|  

In [26]:
#se escribe el parquet tabla 6- Agrupado por año, mes, AntecedenteEnfermedadHuerfana y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/6.SEGCOVID_Tabla_anio_mes_AntecedenteEnfermedadHuerfana_Todos.parquet") 


# 7. Agrupado por año, mes , AntecedenteEPOC y MunicipioAtencion

In [28]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteEPOC
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteEPOC','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+---------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteEPOC|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+---------------+--------------------+------------------+--------------------+
|2021|  1|             No|     47551 - Pivijay|                64|                 234|
|2021|  7|             No|    68498 - Ocamonte|                 5|                  20|
|2022|  9|             No|68081 - Barrancab...|                32|                  71|
|2021| 10|             No| 68001 - Bucaramanga|               851|                5160|
|2022|  6|             No| 68001 - Bucaramanga|               738|                1724|
|2022|  4|             No|    -1 - No Definido|              1923|                2397|
|2021|  6|             No|     91001 - Leticia|                36|                 115|
|2021|  3|             No|       15001 - Tunja|               451|                2551|
|2021| 12|             No|   700

In [29]:
#se escribe el parquet tabla 7- Agrupado por año, mes, AntecedenteEPOC y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/7.SEGCOVID_Tabla_anio_mes_AntecedenteEPOC_Todos.parquet") 

# 8. Agrupado por año, mes, AntecedenteERC y MunicipioAtencion

In [30]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteERC
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteERC','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+--------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteERC|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------+--------------------+------------------+--------------------+
|2021|  5|            No|      41078 - Baraya|                 9|                  29|
|2021| 10|            Sí|   25377 - La Calera|                 1|                  12|
|2021|  9|            No|      73770 - Suárez|                 1|                   4|
|2021|  1|            No|     17877 - Viterbo|                86|                 407|
|2021|  2|            No|     05376 - La Ceja|               272|                1175|
|2021|  1|            No| 86885 - Villagarzón|                90|                 254|
|2021|  5|            No|     05197 - Cocorná|                31|                  60|
|2020|  6|            No|52835 - San Andre...|                25|                  53|
|2021|  4|            No|       76845 - Ull

In [31]:
#se escribe el parquet tabla 8- Agrupado por año, mes, AntecedenteERC y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/8.SEGCOVID_Tabla_anio_mes_AntecedenteERC_Todos.parquet") 

# 9. Agrupado por año, mes, AntecedenteObesidad y MunicipioAtencion

In [32]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteObesidad
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteObesidad','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-------------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteObesidad|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+--------------------+------------------+--------------------+
|2021|  5|                 No|      41078 - Baraya|                 9|                  29|
|2021| 10|                 Sí|   25377 - La Calera|                 3|                  19|
|2021|  1|                 No|     17877 - Viterbo|                71|                 313|
|2021|  2|                 No|     05376 - La Ceja|               251|                1055|
|2021|  1|                 No| 86885 - Villagarzón|                87|                 236|
|2021|  5|                 No|     05197 - Cocorná|                31|                  70|
|2020|  6|                 No|52835 - San Andre...|                22|                  46|
|2021|  4|                 No|       76845 - Ulloa|                14|          

In [33]:
#se escribe el parquet tabla 9- Agrupado por año, mes, AntecedenteObesidad y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/9.SEGCOVID_Tabla_anio_mes_AntecedenteObesidad_Todos.parquet")  

# 10. Agrupado por año, mes, AntecedenteVIH y MunicipioAtencion

In [34]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteVIH
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteVIH','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+--------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteVIH|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------+--------------------+------------------+--------------------+
|2021|  1|            No|     47551 - Pivijay|                63|                 234|
|2021|  1|            No|     17877 - Viterbo|                88|                 412|
|2021|  1|            No|85325 - San Luis ...|                 9|                  30|
|2023|  1|            No|       76869 - Vijes|                 4|                   5|
|2022|  4|            No|    -1 - No Definido|              1924|                2345|
|2022|  6|            No| 68001 - Bucaramanga|               763|                1791|
|2022|  6|            No|     15238 - Duitama|                30|                  93|
|2021| 10|            No| 68001 - Bucaramanga|               872|                5321|
|2021|  4|            No|        05044 - An

In [35]:
#se escribe el parquet tabla 10 - Agrupado por año, mes, AntecedenteVIH y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/10.SEGCOVID_Tabla_anio_mes_AntecedenteVIH_Todos.parquet") 

# Tablas Factores 

# 11. Agrupado por año, mes, FactorRiesgoCancer y MunicipioAtencion

In [13]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoCancer
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoCancer','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoCancer|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+------------------+--------------------+------------------+--------------------+
|2021|  1|                No|     47551 - Pivijay|                65|                 248|
|2021|  1|                No| 86885 - Villagarzón|                91|                 263|
|2021|  7|                No|    68498 - Ocamonte|                 5|                  20|
|2022|  9|                No|68081 - Barrancab...|                33|                  73|
|2021| 10|                No| 68001 - Bucaramanga|               877|                5410|
|2022|  6|                No| 68001 - Bucaramanga|               747|                1749|
|2021|  6|                No|     91001 - Leticia|                37|                 117|
|2021|  3|                No|       15001 - Tunja|               472|                2688|

In [14]:
#se escribe el parquet tabla 11 - Agrupado por año, mes, FactorRiesgoCancer y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/11.SEGCOVID_Tabla_anio_mes_FactorRiesgoCancer_Todos.parquet")

# 12. Agrupado por año, mes,FactorRiesgoCondicionImpidaAislamiento y MunicipioAtencion

In [15]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoCondicionImpidaAislamiento
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoCondicionImpidaAislamiento','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+--------------------------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoCondicionImpidaAislamiento|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------------------------------+--------------------+------------------+--------------------+
|2021|  1|                                    No| 86885 - Villagarzón|                92|                 268|
|2020|  6|                                    No|52835 - San Andre...|                25|                  53|
|2021|  2|                                    No|     05376 - La Ceja|               279|                1225|
|2020| 11|                                    No|25843 - Villa De ...|                30|                  82|
|2022|  2|                                    No|      68770 - Suaita|                14|                  24|
|2022|  4|                                    No|    -1 - No Definido|              1958|                2435|
|

In [16]:
#se escribe el parquet tabla 12 - Agrupado por año, mes, FactorRiesgoCondicionImpidaAislamiento y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/12.SEGCOVID_Tabla_anio_mes_FactorRiesgoCondicionImpidaAislamiento_Todos.parquet")

# 13 Agrupado por año, mes,FactorRiesgoDiabetes y MunicipioAtencion

In [17]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoDiabetes
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoDiabetes','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+--------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoDiabetes|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------------+--------------------+------------------+--------------------+
|2021|  2|                  Sí|   76670 - San Pedro|                 1|                   1|
|2021|  1|                  No|     47551 - Pivijay|                64|                 240|
|2021|  1|                  Sí|     05321 - Guatape|                 1|                   3|
|2021|  1|                  No|     17877 - Viterbo|                85|                 390|
|2023|  1|                  No|       76869 - Vijes|                 4|                   5|
|2022|  4|                  No|    -1 - No Definido|              1978|                2480|
|2022|  6|                  No| 68001 - Bucaramanga|               731|                1700|
|2022|  6|                  No|     15238 - Duitama|                29

In [18]:
#se escribe el parquet tabla 13 - Agrupado por año, mes, FactorRiesgoDiabetes y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/13.SEGCOVID_Tabla_anio_mes_FactorRiesgoDiabetes_Todos.parquet")

# 14 Agrupado por año, mes, FactorRiesgoEnfermedadCardiovascular y MunicipioAtencion

In [19]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoEnfermedadCardiovascular
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoEnfermedadCardiovascular','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+------------------------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoEnfermedadCardiovascular|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+------------------------------------+--------------------+------------------+--------------------+
|2021|  1|                                  No|     47551 - Pivijay|                61|                 206|
|2021|  1|                                  Sí|     05321 - Guatape|                 2|                   5|
|2021|  1|                                  No|     17877 - Viterbo|                73|                 312|
|2021|  1|                                  No|85325 - San Luis ...|                 9|                  30|
|2023|  1|                                  No|       76869 - Vijes|                 4|                   5|
|2022|  4|                                  No|    -1 - No Definido|              1940|                2346|
|2022|  6|         

In [20]:
#se escribe el parquet tabla 14 - Agrupado por año, mes, FactorRiesgoEnfermedadCardiovascular y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/14.SEGCOVID_Tabla_anio_mes_FactorRiesgoEnfermedadCardiovascular_Todos.parquet")

# 15 Agrupado por año, mes, FactorRiesgoEnfermedadRespiratoriaCronica y MunicipioAtencion

In [21]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoEnfermedadRespiratoriaCronica
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoEnfermedadRespiratoriaCronica','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------------------------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoEnfermedadRespiratoriaCronica|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------------------------------------+--------------------+------------------+--------------------+
|2021|  1|                                       No|     47551 - Pivijay|                64|                 237|
|2021|  1|                                       No| 86885 - Villagarzón|                93|                 268|
|2021|  7|                                       No|    68498 - Ocamonte|                 5|                  20|
|2022|  9|                                       No|68081 - Barrancab...|                33|                  73|
|2021| 10|                                       No| 68001 - Bucaramanga|               865|                5296|
|2022|  6|                                       No| 68001 - Bucaramanga|               

In [22]:
#se escribe el parquet tabla 15 - Agrupado por año, mes, FactorRiesgoEnfermedadRespiratoriaCronica y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/15.SEGCOVID_Tabla_anio_mes_FactorRiesgoEnfermedadRespiratoriaCronica_Todos.parquet")

# 16 Agrupado por año, mes,FactorRiesgoInmunodeficiencia y MunicipioAtencion

In [23]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoInmunodeficiencia
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoInmunodeficiencia','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoInmunodeficiencia|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------------------------+--------------------+------------------+--------------------+
|2021|  1|                           No|     47551 - Pivijay|                67|                 233|
|2021|  1|                           No|     17877 - Viterbo|                85|                 399|
|2021|  1|                           No|85325 - San Luis ...|                 9|                  30|
|2023|  1|                           No|       76869 - Vijes|                 4|                   5|
|2022|  4|                           No|    -1 - No Definido|              1993|                2488|
|2022|  6|                           No| 68001 - Bucaramanga|               760|                1787|
|2022|  6|                           No|     15238 - Duitama|                31|  

In [24]:
#se escribe el parquet tabla 16 - Agrupado por año, mes, FactorRiesgoInmunodeficiencia y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/16.SEGCOVID_Tabla_anio_mes_FactorRiesgoInmunodeficiencia.parquet")

# 17 Agrupado por año, mes,FactorRiesgoMayor59 y MunicipioAtencion

In [25]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoMayor59
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoMayor59','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoMayor59|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+--------------------+------------------+--------------------+
|2021|  5|                 No|      41078 - Baraya|                 7|                  26|
|2021|  9|                 No|      73770 - Suárez|                 1|                   4|
|2021|  1|                 No|     17877 - Viterbo|                66|                 274|
|2021|  2|                 No|     05376 - La Ceja|               242|                 978|
|2021|  1|                 No| 86885 - Villagarzón|                84|                 236|
|2021|  5|                 No|     05197 - Cocorná|                26|                  50|
|2020|  6|                 No|52835 - San Andre...|                24|                  50|
|2021|  4|                 Sí|       17486 - Neira|                19|          

In [26]:
#se escribe el parquet tabla 17 - Agrupado por año, mes, FactorRiesgoMayor59 y MunicipioAtencion
result_df.write.parquet("hdfs:/analyticsdata/17.SEGCOVID_Tabla_anio_mes_FactorRiesgoMayor59.parquet")

In [27]:
spark.stop()